In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image, ImageDraw, ImageColor
import IPython.display as Ipd
import subprocess

import torch

import sys
sys.path.insert(1, '../data/')
sys.path.insert(1, '../models/')

import image_utils
import model_helper

In [ ]:
# Deal valid data
in_path = '../image_colored/F_pretrained_on_middle_9_coco_valid/'
F_path = '../image_colored/F_pretrained_on_middle_9_coco_valid/'
tg_path = '../image_colored/F_pretrained_on_middle_9_coco_valid/'
H_path = '../image_colored/H_pretrained_lr1e3_coco_vali/'

image_list = [8, 21, 81, 89, 294, 298, 375, 470, 498]

tmp = Image.open(in_path + '0_i.jpg')
w = tmp.width
h = tmp.height
w_b = 4
h_b = 2

# Draw in one graph
dst = Image.new('RGB', (w * len(image_list) + w_b * (len(image_list) - 1) ,
                        h * 4 + 3 * h_b), 'white')

cnt = 0
for num in image_list:
    im1 = Image.open(in_path + str(num) + '_i.jpg')
    im2 = Image.open(H_path + str(num) + '_o.jpg')
    im3 = Image.open(F_path + str(num) + '_o.jpg')
    im4 = Image.open(tg_path + str(num) + '_t.jpg')
    dst.paste(im1, (cnt * w + cnt * w_b, 0))
    dst.paste(im2, (cnt * w + cnt * w_b, h + h_b))
    dst.paste(im3, (cnt * w + cnt * w_b, h * 2 + h_b * 2))
    dst.paste(im4, (cnt * w + cnt * w_b, h * 3 + h_b * 3))
    cnt += 1

dst.save('res_valid.jpg')
dst.show()

In [ ]:
# Deal real data
img_real_path = '../test_img/img_real/'

image_list = [0, 1, 2, 3]

tmp = Image.open(in_path + '0_i.jpg')
w = tmp.width
h = tmp.height
w_b = 4
h_b = 2

# Draw in one graph
dst = Image.new('RGB', (w * len(image_list) + w_b * (len(image_list) - 1) ,
                        h * 4 + 3 * h_b), 'white')

cnt = 0
for num in image_list:
    im1 = Image.open(img_real_path + str(num) + '_i.jpg')
    im2 = Image.open(img_real_path + str(num) + '_b.jpg')
    im3 = Image.open(img_real_path + str(num) + '_f.jpg')
    im4 = Image.open(img_real_path + str(num) + '_o.jpg')
    dst.paste(im1, (cnt * w + cnt * w_b, 0))
    dst.paste(im2, (cnt * w + cnt * w_b, h + h_b))
    dst.paste(im3, (cnt * w + cnt * w_b, h * 2 + h_b * 2))
    dst.paste(im4, (cnt * w + cnt * w_b, h * 3 + h_b * 3))
    cnt += 1

dst.save('res_real.jpg')
dst.show()

In [ ]:
# Download Image
img_path = '../test_img/img/0.jpg'

img_url = 'https://www.mediastorehouse.com/p/164/young-people-grass-abbey-gardens-bury-st-19535149.jpg.webp'
tmp = subprocess.call(['wget', '--output-document=' + img_path, img_url])

# Resize the image so that it is not too large
with Image.open(img_path) as im:
    w, h = im.size[0], im.size[1]
    if h > 1000:
        w = int(w * 1000 / h)
        im = im.resize((w, 1000), Image.ANTIALIAS)
        im.save(img_path)
        

In [ ]:
# Load Image
with Image.open(img_path) as im:
    print('Size　%d * %d' % (im.size[0], im.size[1]))
    im.show()

In [ ]:
# Inference Box
tmp = subprocess.call('../inference_box/infer_box_for_test.sh', shell=True)

In [ ]:
# Read Inference Box
box_path = '../test_img/img_bbox/0.npz'
box_list = image_utils.get_bbx_fusion(box_path)

with Image.open(img_path).convert("RGB") as im:
    draw = ImageDraw.Draw(im)
    
    colors = ['red','green','yellow','white', 'rgb(100, 200, 0)', 'rgb(100, 0, 200)',
              'rgb(0, 100, 200)', 'rgb(150, 50, 0)']

    print('Box_num:', len(box_list))
    width = max(2, int(im.size[1] / 250))
    for i in range(len(box_list)):
        draw.rectangle([box_list[i][0], box_list[i][1],
                        box_list[i][2], box_list[i][3]], outline=colors[i%8], width=width)

    im.show()

In [ ]:
# Load Model

model_name = 'F_trained_on_large_from_pretrained_lr1e3_0'

model, _ = model_helper.get_model(model_name)
model = model.cuda()

# Prepare data
inputs, targets = image_utils.out_fusion_train_data(img_path, box_path)
inputs = [inputs[0].cuda().unsqueeze(dim=0),
          inputs[1].cuda().unsqueeze(dim=0),
          {key:item.unsqueeze(dim=0) for key, item in inputs[2].items()}]
targets = targets.cuda()


In [ ]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def print_output(gray_channel, ab_channel):
    tmp_res_path = '../test_img/results/'
    gray_path = tmp_res_path + 'g.jpg'
    color_path = tmp_res_path + 'c.jpg'
    
    t1 = torch.zeros_like(gray_channel)
    t2 = torch.zeros_like(gray_channel)
    gray_img = torch.cat((gray_channel, t1, t2), dim=0)
    color_img = torch.cat((gray_channel, ab_channel), dim=0)
    image_utils.save_image_from_tensor(gray_path, gray_img.cpu())
    image_utils.save_image_from_tensor(color_path, color_img.cpu())

    gray_mat = Image.open(gray_path)
    color_mat = Image.open(color_path)
    get_concat_h(gray_mat, color_mat).show()

In [ ]:
# Full image results
model.set_no_fusion(True)

outputs = model(inputs)
print_output(inputs[0][0], outputs[0])

model.set_no_fusion(False)

In [ ]:
# Instances Results
outputs = model.instance_outputs(inputs)

for i in range(len(outputs)):
    print_output(inputs[1][0, i: i+1, :, :], outputs[i])

In [ ]:
# Fusion results
outputs = model(inputs)
print_output(inputs[0][0], outputs[0])

In [ ]:
# Full_image model results

model_name = 'H_pretrained_lr1e3'

model, _ = model_helper.get_model(model_name)
model = model.cuda()

# Prepare data
inputs, _ = image_utils.out_full_data(img_path)
inputs = inputs.cuda().unsqueeze(dim=0)

# Output
outputs = model(inputs)
print_output(inputs, outputs[0])

In [ ]:
# Thank you !